<a href="https://colab.research.google.com/github/Junjaee/Project_PinocchiOH/blob/main/Member/SHIN/Naver_Movie_Crawling_v1_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import re
import os
from random import randrange

import requests
from bs4 import BeautifulSoup as bs
import openpyxl
from urllib.request import urlretrieve

import time
from multiprocessing import Pool, Manager

In [56]:
def crawling(start_code, finish_code):
    
    j = 0
    wb = openpyxl.Workbook()
    sheet = wb.active
    sheet.append(["MOVIE_CODE","MOVIE_TITLE","MOVIE_YEAR","MOVIE_RATING", "MOVIE_GENRE", "MOVIE_COUNTRY", "MOVIE_RTIME", "MOVIE_DIRECTOR", "MOVIE_ACTOR", 
                  "MOVIE_AGE10", "MOVIE_AGE20", "MOVIE_AGE30", "MOVIE_AGE40", "MOVIE_AGE50"])

    global is_ok
    is_ok = False

    for i in range(start_code, finish_code):

        MOVIE_CODE = str(i)
        raw = requests.get("https://movie.naver.com/movie/bi/mi/basic.naver?code=" + MOVIE_CODE)
        html = bs(raw.content, 'html.parser')
        movie = html.select("div.article")    
        
        try:
            for a, m in enumerate(movie):
                # print(a, m)
                # print("*"*40)

                # (3-1) 영화제목 수집
                title = m.select_one("h3.h_movie a")
                MOVIE_TITLE = title.text
                # print(MOVIE_TITLE)

                # (3-2) 영화 개봉년도 수집
                rdate = m.select("strong.h_movie2")
                MOVIE_YEAR = rdate[-1].text[-4:]
                # print(MOVIE_YEAR)                

                # (3-3) 영화평점 수집
                score = m.select("div.main_score div.score.score_left div.star_score a em")
                scores = []
                for s in score:
                    scores.append(s.text)                
                MOVIE_RATING = ''.join(scores)
                # print(MOVIE_RATING)       

                # (3-4) 영화장르 수집
                GENRE = m.select("dl.info_spec dd:nth-of-type(1) p span:nth-of-type(1) a")
                # for g in GENRE:
                #     print(g.text)

                # (3-5) 영화 생산 국가
                COUNTRY = m.select("dl.info_spec dd:nth-of-type(1) p span:nth-of-type(2) a")
                # for c in COUNTRY:
                #     print(c.text)

                # (3-6) 영화 상영시간 수집
                rtime = m.select_one("dl.info_spec dd p span:nth-of-type(3)")
                MOVIE_RTIME = rtime.text[:-2] # 분 생략하고 숫자만 출력
                # print(MOVIE_RTIME)

                # (3-7) 영화감독 수집
                DIRECTOR = m.select("dl.info_spec dd:nth-of-type(2) p a")
                # for d in DIRECTOR:
                #     print(d.text)

                # (3-8) 영화배우 수집
                ACTOR = m.select("dl.info_spec dd:nth-of-type(3) p a")
                # for a in ACTOR:
                #     print(a.text)

                # #(3-9) 성별 선호도(남자)
                # male = html.select("#content > div.article > div.mv_info_area > div.mv_info > div.viewing_graph")
                # # print(male)
                # for a in male:
                #     print(a.text)

                # (3-10-1) 연령별 선호도(10대)
                age_ten = m.select("div.bar_graph div:nth-of-type(1) strong")
                MOVIE_AGE10 = age_ten[0].text[:-1]
                # print(MOVIE_AGE10)

                # (3-10-2) 연령별 선호도(20대)
                age_twenty = m.select("div.bar_graph div:nth-of-type(3) strong")
                MOVIE_AGE20 = age_twenty[0].text[:-1]
                # print(MOVIE_AGE20)

                # (3-10-3) 연령별 선호도(30대)
                age_thirty = m.select("div.bar_graph div:nth-of-type(5) strong")
                MOVIE_AGE30 = age_thirty[0].text[:-1]
                # print(MOVIE_AGE30)

                # (3-10-4) 연령별 선호도(40대)
                age_fourty = m.select("div.bar_graph div:nth-of-type(7) strong")
                MOVIE_AGE40 = age_fourty[0].text[:-1]
                # print(MOVIE_AGE40)

                # (3-10-5) 연령별 선호도(50대 이상)
                age_fifty = m.select("div.bar_graph div:nth-of-type(9) strong")
                MOVIE_AGE50 = age_fifty[0].text[:-1]
                # print(MOVIE_AGE50)

                if len(MOVIE_AGE50) > 0:
                    # (7) 영화관련정보 엑셀(xlsx) 형식 저장
                    # (7-1) 데이터 만들기-1 : HTML로 가져온 영화장르/영화감독/영화배우 정보에서 TEXT정보만 뽑아서 리스트 형태로 만들기
                    GENREs = [g.text for g in GENRE]
                    COUNTRYs = [c.text for c in COUNTRY]
                    DIRECTORs = [d.text for d in DIRECTOR]
                    ACTORs = [a.text for a in ACTOR]

                    #(7-2) 데이터 만들기-2 : 여러 개로 이루어진 리스트 형태를 하나의 문자열 형태로 만들기
                    MOVIE_GENRE = ','.join(GENREs)
                    MOVIE_COUNTRY = ','.join(COUNTRYs)
                    MOVIE_DIRECTOR = ','.join(DIRECTORs)
                    MOVIE_ACTOR = ','.join(ACTORs)

                    # (7-3) 영화관련정보 엑셀 행 추가 : line by line 으로 추가하기
                    sheet.append([MOVIE_CODE,MOVIE_TITLE,MOVIE_YEAR,MOVIE_RATING, MOVIE_GENRE, MOVIE_COUNTRY, MOVIE_RTIME, MOVIE_DIRECTOR, MOVIE_ACTOR, 
                                MOVIE_AGE10, MOVIE_AGE20, MOVIE_AGE30, MOVIE_AGE40, MOVIE_AGE50])

                    is_ok = True

            if is_ok == True:
                j = j + 1
            print(str(start_code)[:-4], "번 프로세스 - ", finish_code - start_code, "개중에", finish_code - i - 1, "개 남음")
            print(j, "개의 영화 정보저장 완료")

        except:
            print("에러발생")
            print(str(start_code)[:-4], "번 프로세스 - ", finish_code - start_code, "개중에", finish_code - i - 1, "개 남음")
            continue

        finally:
            print("완료")
            print("*"*40)
            wb.save("navermovie{}.xlsx".format(start_code))

In [43]:
crawling(10001, 10002)
# crawling(10001, 210001)

1 개중에 0 개 남음
1 개의 영화 정보저장 완료
완료
****************************************


In [ ]:
# 코어 20개로 크롤링하기(속도 x20배 빠르게)

def poolcontext(*args, **kwargs):
    pool = multiprocessing.Pool(*args, **kwargs)
    yield pool
    pool.terminate()

if __name__=='__main__':
    start_time = time.time()
    inputs = [10001, 20001, 30001, 40001, 50001, 60001, 70001, 80001, 90001, 100001, 110001, 120001, 130001, 140001, 150001, 160001, 170001, 180001, 190001, 200001]
    outputs = [20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, 110000, 120000, 130000, 140000, 150000, 160000, 170000, 180000, 190000, 200000, 210000]

    pool = Pool(processes=20)
    pool.starmap(crawling, zip(inputs, outputs))
    print("--- %s seconds ---" % (time.time() - start_time))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
완료
****************************************
에러발생
17 번 프로세스 -  9999 개중에 9968 개 남음
완료
****************************************
에러발생
12 번 프로세스 -  9999 개중에 9969 개 남음
완료
****************************************
에러발생
에러발생
8 번 프로세스 -  9999 개중에 9970 개 남음
7 번 프로세스 -  9999 개중에 9972 개 남음
완료
완료
****************************************
****************************************
에러발생
13 번 프로세스 -  9999 개중에 9969 개 남음
완료
에러발생
****************************************
14 번 프로세스 -  9999 개중에 9969 개 남음
완료
****************************************
에러발생
9 번 프로세스 -  9999 개중에 9969 개 남음
완료
****************************************
에러발생
16 번 프로세스 -  9999 개중에 9969 개 남음
에러발생
완료
6 번 프로세스 -  9999 개중에 9957 개 남음
완료
****************************************
****************************************
에러발생
3 번 프로세스 -  9999 개중에 9970 개 남음
완료
****************************************
12 번 프로세스 -  9999 개중에 9968 개 남음
5 개의 영화 정보저장 완료
완료
****************************************
에러발생
4 번 프로세스 -  9999 개

In [ ]:
##################### 성별 선호 데이터 가져오기 -> ajax 크롤링(동적 크롤링) -> 실패

In [ ]:
pip install fake_useragent

In [21]:
from urllib.request import urlopen, Request
from fake_useragent import UserAgent
import json

# fake_useragent 모듈을 통한 User-Agent 정보 생성
useragent = UserAgent()
# print(useragent.chrome)
# print(useragent.ie)
# print(useragent.safari)
# print(useragent.random)

# 헤더 선언 및 referer, User-Agent 전송
headers = {
    'referer' : 'https://movie.naver.com/movie/bi/mi/basic.naver?code=10001',
    'User-Agent' : useragent.chrome
}

# 주식 데이터 요청 URL
url = 'https://movie.naver.com/movie/bi/mi/basic.naver?code=10001'

# 주식 데이터 요청
# response = urlopen(Request(url, headers=headers))

req = requests.get(url, headers = headers)



# 응답 데이터 str 타입을 json 포맷으로 변환 및 data 저장
# print(response)


# rank_json = json.loads(response)['aActualGenderPointData']
# print(rank_json)

# print(f"날짜 : {rank_json[0]['date']}, 마켓: {rank_json[0]['market']}")
# for rank in rank_json:
#     print(f"종목 : {rank['sectorName']}, 상승/하강: {rank['change']}, 등락율 : {round(rank['changeRate']*100, 3)}")

In [40]:
# result = json.loads(req.text)

# print(result)


# print(response)
print(req.text)

# rank_json = json.loads(response)
# print(rank_json)










	
	
		
		
			
						
			
				
				
					
				
				 
			
		
		
			
		
		
	
	
	
	
	

<!DOCTYPE html>
<html lang="ko">
<head>
<meta charset="utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta http-equiv="imagetoolbar" content="no">
<title>시네마 천국 : 네이버 영화</title>

	
	
	
	
	
		
		
	



<meta property="me2:image" content="https://ssl.pstatic.net/imgmovie/today/naverme/naverme_profile.jpg"/>
<meta property="me2:post_tag" content="네이버영화 시네마 천국"/>
<meta property="me2:category1" content="네이버영화"/>
<meta property="me2:category2" content="시네마 천국"/>



	
	
	
		<meta property="og:title" content="시네마 천국"/>
	


<meta property="og:type" content="article"/>

<meta property="og:url" content="https://movie.naver.com/movie/bi/mi/basic.naver?code=10001"/>


	
		<meta property="og:image" content="https://movie-phinf.pstatic.net/20210409_134/16179547654813MHq5_JPEG/movie_image.jpg?type=m665_443_2"/>
	
	


	
	<meta property="og:description" content="어린 시절 영화가 세상의 전부였던 소년 토토는 학교 수업을 마치면

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13503 sha256=f396974fa213829377bb33ccf53495d44133980aab71002109fc485d4fa920bf
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent
